In [1]:
!pip install langchain



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


LLM application with LCEL

In [2]:
# open source API key 
import os 
from dotenv import load_dotenv
load_dotenv() 

# either openai or groq can be used as the key 

# import openai
# openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
!pip install langchain_groq


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from langchain_groq import ChatGroq
# from langchain_openai import ChatOpenAI

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model 
# we can see the model details in the output 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AB70D2D2E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AB71F74680>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [5]:
# LLM model should basically give reply of the query sent 
# we need to mention behaviours to the model 
# so that it gives proper outputs  

!pip install langchain_core


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from langchain_core.messages import HumanMessage, SystemMessage
#  used to specify which is the instruction is given by the user as query
# and which is the instruction for the LLM machine 

messages=[
    SystemMessage(content="Translate the following from Eng to French"),
    HumanMessage(content="i live in indore")
]
response= model.invoke(messages)
# invoke msg will generate an output starting iwth AIMessage that means
# it is a response from LLM model with the translated output & meta data


In [20]:
response

AIMessage(content="J'habite à Indore. \n", response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 26, 'total_tokens': 37, 'completion_time': 0.022, 'prompt_time': 0.002788247, 'queue_time': None, 'total_time': 0.024788247}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-20ece401-e1d2-4dc0-adfa-e918f4d90efe-0', usage_metadata={'input_tokens': 26, 'output_tokens': 11, 'total_tokens': 37})

In [18]:
# output parse is responsible to display the output content 
from langchain_core.output_parsers import StrOutputParser 

parser = StrOutputParser()
parser.invoke(response)


'मुझे इंदौर में रहना है। (Mujhe Indore mein rahna hai.) \n'

In [9]:
# using LCEL to chain the components 
chain = model|parser        #chain of outputs is passed 
chain.invoke(messages)

'Here\'s the translation:\n\n**French:**\n\n* Je vis à Indore.\n\n**Hindi:**\n\n* मैं इंदौर में रहता/रहती हूँ (Main Indore mein rehta/rehti hoon)\n\n**Notes:**\n\n* The Hindi translation uses the verb "रहता" (rehta) for male speakers and "रहती" (rehti) for female speakers. \n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [10]:
# prompt templates 
from langchain_core.prompts import ChatPromptTemplate

generic_temp = "Translate the following from {input_language} to {output_lang}:"

# this is the basic prompt 
prompt= ChatPromptTemplate.from_messages(
    [("system", generic_temp) , ("user", "{text}")
    ]
)

In [11]:
response= prompt.invoke({"lang":"French" , "text":"Hello"})
# res2 = prompt.invoke({"lang":"Hindi" , "text":"Hello"})

In [12]:
response.to_messages() 
# res2.to_messages()


[SystemMessage(content='Translate the following into French:'),
 HumanMessage(content='Hello')]

In [13]:
chain= prompt | model | parser
chain.invoke({"lang":"French", "text":"Hello"})


'Bonjour  \n'

Langserve is used to convert as RESAPI